# Quantifiers

The quantifiers are parameters calculated for each simulation that allows us to evaluate the characteristics of the colonie. The main characteristic are the four-cell organisation and the elongation of the colony.

The chosen quantifiers are :
- the aspect ratio $\alpha_r$.
- the density $\delta$.
- $s$ the mean of the difference between the distance the rest length of two disks.
- the distance $d_2$.

## Imports

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from PIL import Image
from skimage import measure
from skimage.io import imread
from skimage.morphology import convex_hull_image
from skimage import data, img_as_float
from skimage.util import invert
from scipy.ndimage import binary_fill_holes
from numpy.linalg import norm

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../')
import plot_model as pm

## Data loading and cleaning

In [20]:
file = "simu5"
def convert(L):
    T = L.split(" ")
    T.pop()
    return list(np.array(T,dtype=float))

df = pd.read_csv(file+".txt",sep="\t")

df["X"]=df["X"].apply(convert)
# df

FileNotFoundError: [Errno 2] No such file or directory: 'simu5.txt'

## 1. Calculating the aspect ratio

The aspect ratio refers to the elongation of the shape of the colony.
To calculate $\alpha_r$, we need to calculate the convex hull of the points of the colony and fit an ellipse on it. $\alpha_r$ is then defined as the ratio $\frac{l}{L}$ with $l$ the semi-minor axis and $L$ the semi-major axis.
<br>The aspect ratio equal 1 if the colony form a perfect circle

In [ ]:
# Merge all the X datas
X = []
dfX = df["X"]
for i in range(dfX.shape[0]):
    X = X + dfX[i]

# Separates x and y
x=X[0::2]
y=X[1::2]

# Converting the points to a compatible format for the Convexhull class
points = np.array([x,y])
points = np.transpose(points)

# Calculating the hull
hull = ConvexHull(points)

# Calculating the vertices of the hull
x1 =[]
y1 =[]
for i in hull.vertices:
    x1.append(x[i])
    y1.append(y[i])

plt.figure(figsize=(10,8))
plt.plot(points[:,0],points[:,1],".",color="grey",markersize=20)
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], '+-',)
plt.title("Bacteria colony and the convex hull")
plt.show()

In [ ]:
# reference : https://scipython.com/blog/direct-linear-least-squares-fitting-of-an-ellipse/

def fit_ellipse(x, y):
    """

    Fit the coefficients a,b,c,d,e,f, representing an ellipse described by
    the formula F(x,y) = ax^2 + bxy + cy^2 + dx + ey + f = 0 to the provided
    arrays of data points x=[x1, x2, ..., xn] and y=[y1, y2, ..., yn].

    Based on the algorithm of Halir and Flusser, "Numerically stable direct
    least squares fitting of ellipses'.


    """

    D1 = np.vstack([x**2, x*y, y**2]).T
    D2 = np.vstack([x, y, np.ones(len(x))]).T
    S1 = D1.T @ D1
    S2 = D1.T @ D2
    S3 = D2.T @ D2
    T = -np.linalg.inv(S3) @ S2.T
    M = S1 + S2 @ T
    C = np.array(((0, 0, 2), (0, -1, 0), (2, 0, 0)), dtype=float)
    M = np.linalg.inv(C) @ M
    eigval, eigvec = np.linalg.eig(M)
    con = 4 * eigvec[0]* eigvec[2] - eigvec[1]**2
    ak = eigvec[:, np.nonzero(con > 0)[0]]
    return np.concatenate((ak, T @ ak)).ravel()

def cart_to_pol(coeffs):
    """

    Convert the cartesian conic coefficients, (a, b, c, d, e, f), to the
    ellipse parameters, where F(x, y) = ax^2 + bxy + cy^2 + dx + ey + f = 0.
    The returned parameters are x0, y0, ap, bp, e, phi, where (x0, y0) is the
    ellipse centre; (ap, bp) are the semi-major and semi-minor axes,
    respectively; e is the eccentricity; and phi is the rotation of the semi-
    major axis from the x-axis.

    """

    # We use the formulas from https://mathworld.wolfram.com/Ellipse.html
    # which assumes a cartesian form ax^2 + 2bxy + cy^2 + 2dx + 2fy + g = 0.
    # Therefore, rename and scale b, d and f appropriately.
    a = coeffs[0]
    b = coeffs[1] / 2
    c = coeffs[2]
    d = coeffs[3] / 2
    f = coeffs[4] / 2
    g = coeffs[5]

    den = b**2 - a*c
    if den > 0:
        raise ValueError('coeffs do not represent an ellipse: b^2 - 4ac must'
                         ' be negative!')

    # The location of the ellipse centre.
    x0, y0 = (c*d - b*f) / den, (a*f - b*d) / den

    num = 2 * (a*f**2 + c*d**2 + g*b**2 - 2*b*d*f - a*c*g)
    fac = np.sqrt((a - c)**2 + 4*b**2)
    # The semi-major and semi-minor axis lengths (these are not sorted).
    ap = np.sqrt(num / den / (fac - a - c))
    bp = np.sqrt(num / den / (-fac - a - c))

    # Sort the semi-major and semi-minor axis lengths but keep track of
    # the original relative magnitudes of width and height.
    width_gt_height = True
    if ap < bp:
        width_gt_height = False
        ap, bp = bp, ap

    # The eccentricity.
    r = (bp/ap)**2
    if r > 1:
        r = 1/r
    e = np.sqrt(1 - r)

    # The angle of anticlockwise rotation of the major-axis from x-axis.
    if b == 0:
        phi = 0 if a < c else np.pi/2
    else:
        phi = np.arctan((2.*b) / (a - c)) / 2
        if a > c:
            phi += np.pi/2
    if not width_gt_height:
        # Ensure that phi is the angle to rotate to the semi-major axis.
        phi += np.pi/2
    phi = phi % np.pi

    return x0, y0, ap, bp, e, phi

def get_ellipse_pts(params, npts=100, tmin=0, tmax=2*np.pi):
    """
    Return npts points on the ellipse described by the params = x0, y0, ap,
    bp, e, phi for values of the parametric variable t between tmin and tmax.

    """

    x0, y0, ap, bp, e, phi = params
    # A grid of the parametric variable, t.
    t = np.linspace(tmin, tmax, npts)
    x = x0 + ap * np.cos(t) * np.cos(phi) - bp * np.sin(t) * np.sin(phi)
    y = y0 + ap * np.cos(t) * np.sin(phi) + bp * np.sin(t) * np.cos(phi)
    return x, y

x = np.array(x1)
y = np.array(y1)

# Calculating the Cartesian coefficients
coeffs = fit_ellipse(x, y)
print('a, b, c, d, e, f =', coeffs)

# Converting the coefficient into ellipse parameters
x0, y0, ap, bp, e, phi = cart_to_pol(coeffs)
print('x0, y0, ap, bp, e, phi = ', x0, y0, ap, bp, e, phi)

plt.figure(figsize=(10,8))
plt.plot(x, y, 'x')     # given points
x, y = get_ellipse_pts((x0, y0, ap, bp, e, phi))    # Ellipse
plt.plot(x, y)
plt.title("Ellipse fitting")
plt.show()

print("The aspect ratio is : ",bp/ap)

## 2. Calculating the density

The density of the colony can be calculated by using image analysis tool.
The principle is to calculate the number of pixels that represents the bacteria and the number of pixels of the filled colony. The density is then the ration betaween these two values.
<br> The density equal 1 if the bacteria occupy all the space of the colony.

In [ ]:
# Creating the grey and black image
pm = pm.Plot(file + ".txt")
pm.save()

# Loading and displaying the image
image_path = file + ".png"
Img_array = imread(image_path)
plt.figure(figsize=(10,8))
plt.imshow(Img_array)
plt.show()

shape = Img_array.shape

def identify_bact(tab):
    """Check if the pixel is part of a bacterium"""
    if tab[0]==0 and tab[1]==0 and tab[2]==0 :
        return True

# Calculating the number of pixels of the bacterias
nb_color = 0
for i in range(shape[0]):
    for j in range(shape[1]):
        if identify_bact(Img_array[i,j,]):
            nb_color+=1

# Inversing the colors
Img_array = imread(image_path,as_gray=True)
im =invert(Img_array)

# Converting of array to binary
def gray_to_bin(im_array):
    """ Convert the image in greyscale to binary"""
    for i in range(im_array.shape[0]):
        for j in range(im_array.shape[1]):
            if im_array[i,j]!=1:
                im_array[i,j]=0

gray_to_bin(im)

# Filling the holes
filled = binary_fill_holes(im)
plt.figure(figsize=(10,8))
plt.imshow(filled,cmap=plt.cm.gray)
plt.show()

# Calculating the area of the filled colony
nb = 0
shape = filled.shape
for i in range(shape[0]):
    for j in range(shape[1]):
        if filled[i,j]==True:
            nb+=1

print("density=",nb_color/nb)

## 3. The length quantifier

The length quantifier quantifies the state of the hookean springs compared to their rest state.
<br>More $s$ is near to 0, more the springs are close to their equilibrum.

In [ ]:
# Disks radius
l = df["Disks radius"][0]

# Fonction to apply 
def qlength(X,l):
    """ Calculates the length quantifier """
    s = 0
    p_i = len(X)

    for i in range(0,p_i-3,2):
        s+= np.abs(norm(np.array([X[i+2],X[i+3]]) - np.array([X[i],X[i+1]]))-l)
    return s/(p_i-1)

# Calculating the mean for each bacterium
mean_i = df["X"].apply(qlength,args=(l,))

# Calculating the mean for all bacteria
Ql = mean_i.mean()
print("Length quantifier = ",Ql)